#의사결정 트리 (스무고개)

##목표

>의사결정 트리 소개

>데이터 집합에서 일관성 측정

>의사결정 트리 구축을 위한 반복사용

>메스플롯라이브러리에서 트리 사용

#가장 많이 사용되는 기술

장점: 계산 비용 적음 / 학습 결과 이해 쉬움 / 누락된 값 있어도 처리 가능 / 분류와 관련없는 속성이 있어도 가능

단점: 과적합이 되기 쉬움

적용: 수치형 값, 명목형 값

#의사결정 트리의 일반적인 접근방법
1. 수집: 모든 방법
2. 준비: 명목형 값만 처리 가능하기 때문에, 연속형은 양자화 되야함
3. 분석: 모든 방법, 트리를 만든 후에 시각적 트리 검토
4. 훈련: 트리 형태로 데이터 구축
5. 검사: 학습된 트리를 활용하여 오류율
6. 사용: 모든 지도학습에서 사용될 수 있음, 종종 데이터를 더 잘 다루기위해..

http://javacan.tistory.com/entry/MachineLearningInAction-03-DecisitionTree-ID3

In [1]:
from math import log
import operator

'''
해양 동물 데이터
Col1:지표면에 닿지 않고 살아남을수 있는가?
Col2:지느러미 보유?
Col3:물고기 or not?

'''

def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing','flippers']
    #change to discrete values
    return dataSet, labels

<img src="entropy.png">

In [4]:
'''
정보이득 (Information Gain): 데이터를 분할하기 전과 후의 변화
앤트로피(entropy): 데이터 집합에 대한 정보 측정 방법

'''

def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet: #가능한 모든 분류 항목에 대한 딕셔너리 생성
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys(): labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob,2) #밑수가 2인 로그
    return shannonEnt

In [5]:
myDat,labels = createDataSet() #데이터 생성
calcShannonEnt(myDat) #엔트로피 값

0.9709505944546686

In [7]:
#엔트로피가 높은것은 더 혼잡하다는 이야기
#maybe항을 추가하고 판단
myDat[0][-1] = "maybe"
myDat

[[1, 1, 'maybe'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [8]:
calcShannonEnt(myDat)

1.3709505944546687

In [13]:
#데이터 집합 분할하기
#3.2 주어진 속성을 데이터 분할하기

#예제 데이터
a=[1,2,3]
b=[4,5,6]
#a.append(b)

a.extend(b)
a

[1, 2, 3, 4, 5, 6]

In [15]:

def splitDataSet(dataSet, axis, value):
    retDataSet = [] # 분할 리스트 생성
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]     #분할한 속성 잘라내기
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet   

In [17]:
#splitDataSet(myDat, 0, 1)
splitDataSet(myDat, 0, 0)

[[1, 'no'], [1, 'no']]

In [18]:
#데이터를 분할한 가장 좋은 속성 선택하기

def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1      #the last column is used for the labels
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0; bestFeature = -1
    for i in range(numFeatures):        #iterate over all the features
        featList = [example[i] for example in dataSet]#분류 항목 표시에 대해 중복이 없는 리스트 생성
        uniqueVals = set(featList)       #get a set of unique values
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value) #각각 분할을 위한 엔트로피 계산
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)     
        infoGain = baseEntropy - newEntropy     #calculate the info gain; ie reduction in entropy
        if (infoGain > bestInfoGain):       #가장 큰 정보이득 찾기
            bestInfoGain = infoGain         #if better than current best, set to best
            bestFeature = i
    return bestFeature                      #returns an integer

In [19]:
chooseBestFeatureToSplit(myDat)
myDat

[[1, 1, 'maybe'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [20]:
#재귀적 트리 생성
#다수결 원칙
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

#트리 생성
def createTree(dataSet,labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList): 
        return classList[0]#모든 분류 항목이 같을 때 멈춤
    if len(dataSet[0]) == 1: #속성이 더 이상 업을 때 가장 많은 수를 반환
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat]) # 유일한 값의 리스트를 구함
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]       #copy all of labels, so trees don't mess up existing labels
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels)
    return myTree                        

In [21]:
myTree = createTree(myDat, labels)
myTree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'maybe'}}}}

In [22]:
def classify(inputTree,featLabels,testVec):
    firstStr = inputTree.keys()[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    key = testVec[featIndex]
    valueOfFeat = secondDict[key]
    if isinstance(valueOfFeat, dict): 
        classLabel = classify(valueOfFeat, featLabels, testVec)
    else: classLabel = valueOfFeat
    return classLabel

def storeTree(inputTree,filename):
    import pickle
    fw = open(filename,'w')
    pickle.dump(inputTree,fw)
    fw.close()
    
def grabTree(filename):
    import pickle
    fr = open(filename)
    return pickle.load(fr)